In [1]:
import os, serial, time
from numpy import *
from colorsys import *

# Communication functions

In [2]:
def updateLed( sObj, dat=None, channel=0 ):
    if dat is None:
        nLeds = 1024
        ledDat = bytearray( os.urandom( nLeds*3 ) )
    elif type(dat) is int:
        nLeds = dat
        ledDat = bytearray( os.urandom( nLeds*3 ) )
    else:
        nLeds = len(dat)
        ledDat = bytearray( dat )
    sDat = bytes("LED {0} {1}\n".format(channel, len(ledDat)), "utf8") + ledDat
    sObj.write( sDat )

# Open serial port connection

In [45]:
s.close()

In [46]:
try:
    s.close()
except:
    pass
s = serial.Serial("/dev/ttyACM0", 115200, timeout=1)

# Get ID and Software Version

In [47]:
s.read_all()          #Clear receive buffer
s.write(b"\n*IDN?\n") #First \n clears send buffer
s.read_until()

b'ID:MB:V0.2\n'

# Get Switch state

In [48]:
#%%timeit   #1000 loops, best of 3: 1.78 ms per loop
s.write(b"SW?\n")
s.read_until()

b'SW:f6ffffffffffffff0000000000000000000000000000000000000000000000000000000000000000\n'

# Setup low speed WS2811 LED strand

In [121]:
s.write(b"LEC 1 3200000\n")

14

Datasheet spec. would be **1600000 Hz**, but my LED strand glitches like hell with that setting. **1601000 Hz** works perfectly though.

# Setup high speed WS2812 LED strip

In [50]:
s.write(b"LEC 0 3200000\n")

14

# Benchmark LED throughput (worst case)

In [113]:
%%timeit
x = array( ones(1024*3), dtype=uint8)
updateLed(s, x, 0)
updateLed(s, x, 1)
updateLed(s, x, 2)
# [Only USB comm.] 10 loops, best of 100: 36.0 ms per loop (28 Hz)
# [USB + SPI send] 10 loops, best of 100: 45.5 ms per loop (22 Hz)

10 loops, best of 3: 72.4 ms per loop


It looks like the USB communication is the bottleneck at the moment. Note that the firmware can transmit on several channels simultaneously but will block if a channel is updated, which has not finished transmitting yet.

**TLDR:** If you need > 30 Hz refresh rate, do not connect more than 512 LEDs per string

# Play with LEDs

### Glitch test

Do many sets of the same values. LEDs should not flicker at all!

In [137]:
x = zeros( 59*3, dtype=uint8 )
x[0:-1][::2] = 1
#x[-6:] = 1
for i in range(8000):
    updateLed( s, x, 0 )
    updateLed( s, x, 1 )
    updateLed( s, x, 2 )

In [133]:
s.read_all()

b''

### Turn ON one color after another

In [136]:
updateLed(s, zeros(1024*3, dtype=uint8), 2)

In [129]:
while(True):
    x = zeros( 59*3, dtype=uint8 )
    for i in range(len(x)):
        x[i] = 25
#         updateLed(s, x, 0)
        updateLed(s, x, 1)
#         updateLed(s, x, 2)
        time.sleep(1/60)

KeyboardInterrupt: 

### All LEDs same color

In [38]:
x = array( [0, 0, 0]*59, dtype=uint8 )
updateLed( s, x, 0 )
updateLed( s, x, 1 )
updateLed( s, x, 2 )

### Gamma corrected fade-UP

In [14]:
# Precompute quadratic brightness values
bVals = array( arange(33)**2/4, dtype=uint8 )
bVals[-1] = 255

In [16]:
for bVal in bVals:
    x = array( [bVal, bVal, bVal]*5, dtype=uint8 )
    updateLed( s, x, 0 )
    time.sleep(0.03)

### Each LED a random color

In [40]:
# Set it once
x = array(random.randint(0,255,58*3),dtype=uint8)
updateLed(s, x, 0)
updateLed(s, x, 1)
updateLed(s, x, 2)

KeyboardInterrupt: 

In [52]:
# Set it in a loop
while True:
    x = array(random.randint(0,2,58*3)*25,dtype=uint8)
    updateLed( s, x, 0 )
    updateLed( s, x, 1 )
    updateLed( s, x, 2 )
    time.sleep(0.05)

KeyboardInterrupt: 

# Rainbow

### Darken LEDs

In [229]:
s.write(b"LEC 1 1601000\n")
updateLed( s , zeros(70*3, dtype=uint8), 0 )
updateLed( s , zeros(70*3, dtype=uint8), 1 )
updateLed( s , zeros(70*3, dtype=uint8), 2 )

### Setup color values

In [78]:
nCols = 200
x = arange(nCols)/(nCols-1)
x2 = list(map( hsv_to_rgb, x, ones(nCols), ones(nCols)*1.0))
rawOut = array( array(x2) * 255, dtype=uint8 )

### Roll color values through the string

In [ ]:
while(True):
    updateLed( s, array(rawOut[:60]/5, dtype=uint8), 0 )
    updateLed( s, rawOut[:60], 1 )
    updateLed( s, rawOut[:60], 2 )
    rawOut = roll(rawOut, 3)
    time.sleep(1/60)

# Enable reporting of Switch Events

In [7]:
#SWE   : <OnOff> En./Dis. reporting of switch events.
s.write(b"SWE 1\n")

6

# Play with Relay Outputs

In [1]:
try:
    s.close()
except:
    pass

In [2]:
s = serial.Serial( "/dev/ttyACM0", timeout=1 )

NameError: name 'serial' is not defined

### Flush serial input buffer

In [10]:
print( s.read_all() )
s.write(b"\n*IDN?\n")
print( s.read_until() )

b''
b'ID:MB:V0.1\n'


### Set a Solenoid to a specific power

In [220]:
#OUT   : <hwIndex> <PWMlow>
s.write(b"OUT 0x40 2\n")

11

### Trigger a solenoid pulse

In [11]:
#OUT   : <hwIndex> <PWMlow> [tPulse] [PWMhigh]
s.write(b"OUT 0x40 0 20 2\n")
s.write(b"OUT 0x41 0 20 2\n")
s.write(b"OUT 0x42 0 20 2\n")

16

### Mushrooms

In [14]:
for i in (0x40, 0x41, 0x42):
    s.write( "OUT {0} 0 30 2\n".format(i).encode("utf8") )
    time.sleep( 0.1 )
s.write( b"OUT 0x3D 0 30 2000\n" )

19

### Flippers

In [15]:
for i in (0x3C, 0x3E):
    s.write( "OUT {0} 0 50 2000\n".format(i).encode("utf8") )
    time.sleep( 0.1 )

# Setup quick-fire rules

Trigger Second Solenoid when Second Switch closes

In [136]:
#RUL <ID> <IDin> <IDout> <trHoldOff> <tPulse> <pwmOn> <pwmOff> <bPosEdge>
s.write(b"RUL 0 0x0000 64 30 20 5 0 1\n")

28

First switch --> First Solenoid. Add a 1000 ms hold-off time (triggers max. once a second). 

In [549]:
s.write(b"RUL 1 0x0000 64 1000 100 5 0 1\n")

31

### Disable the rules

In [18]:
for i in range(16):
    s.write("RULE {0} 0\n".format(i).encode("UTF8"))

### Rules for basic Flipper operation (Fan-Tas-Tic)

In [29]:
#RUL    <ID><IDin><IDout><trHoldOff><tPulse><pwmOn><pwmOff><bPosEdge>
# Note that buttons are active low
# Flipper rules
# Attack + Hold on neg. edge
s.write(b"RUL 0 0x4C 0x3C 200 75 3000 1000 0\n")
s.write(b"RUL 1 0x4F 0x3E 200 75 3000 1000 0\n")
# Release on pos. edge
s.write(b"RUL 2 0x4C 0x3C 0 0 0 0 1\n")
s.write(b"RUL 3 0x4F 0x3E 0 0 0 0 1\n")
# Jet bumper rules
for rulId, hwIndexIn, hwIndexOut in zip( (4,5,6,7), (0x05, 0x04, 0x03, 0x1A), (0x40, 0x41, 0x42, 0x3D) ):
    if hwIndexOut == 0x3D:
        power = 4500
    else:
        power = 4
    rulStr = "RUL {0} {1} {2} 0 20 {3} 0 1\n".format( rulId, hwIndexIn, hwIndexOut, power )
    print( rulStr )
    s.write( rulStr.encode("UTF8") )
    # disable debouncing for the jet bumper inputs
    #s.write( "DEB {0} 0\n".format(hwIndexIn).encode("UTF8") )
# Captive ball rules
for rulId, hwIndexIn, hwIndexOut in zip( (8,9,10), (0x16, 0x23, 0x00), (0x46, 0x47, 0x43) ):
    rulStr = "RUL {0} {1} {2} 1000 75 3 0 1\n".format( rulId, hwIndexIn, hwIndexOut )
    print( rulStr )
    s.write( rulStr.encode("UTF8") )

    

RUL 4 5 64 0 20 4 0 1

RUL 5 4 65 0 20 4 0 1

RUL 6 3 66 0 20 4 0 1

RUL 7 26 61 0 20 4500 0 1

RUL 8 22 70 1000 75 3 0 1

RUL 9 35 71 1000 75 3 0 1

RUL 10 0 67 1000 75 3 0 1



    #Flippers
    RUL 0 0x4C 0x3C 200 75 3000 500 0
    RUL 1 0x4F 0x3E 200 75 3000 500 0
    RUL 2 0x4C 0x3C 0 0 0 0 1
    RUL 3 0x4F 0x3E 0 0 0 0 1

    #Jet bumpers
    RUL 4 5 64 0 15 4 0 1
    RUL 5 4 65 0 15 4 0 1
    RUL 6 3 66 0 15 4 0 1
    RUL 7 26 61 0 15 4000 0 1
    
    #Captive holes
    #R
    RUL 8 0x16 0x46 500 75 3 0 1
    #L
    RUL 9 0x23 0x47 500 75 3 0 1
    #T
    RUL 10 0x00 0x43 500 75 3 0 1
    

In [31]:
s.close()


In [21]:
s.write(b"SWE 0\n")

6

# Pseudo console

In [30]:
while True:
    rx = s.read_until()
    if len( rx ) > 0:
        print( rx )

KeyboardInterrupt: 